## moduleの読み込み

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pickle
import gc
import mlflow
from tqdm import tqdm


from umap import UMAP
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import optuna.integration.lightgbm as lgb_o
import lightgbm as lgb
import optuna

## 関数の作成

In [ ]:
#データの読み込みからpandasのデータフレームに読み込む
def read_db(name):
    local = "main.db_odl_user_838812."
    df = spark.table(local + name).toPandas()
    return df

#加工したデータをデータベースに保存する
def save_db(df,name:str):
    now = str(datetime.date.today()).replace("-","")
    spark.createDataFrame(df).write.mode("overwrite").option("mergeSchema", "true").saveAsTable(name + "_" +now)

#データを加工する
def process_data(df):
    #数字型の日付は月バラバラで見る
    df['account_open_m'] = df['account_open_date'].astype(str).str[-2:].astype(int)
    df['hl_open_m'] = df['hl_open_date'].astype(str).str[-2:].astype(int)

    #開設日から何週経過しているのかの特徴量を追加
    df["hl_elapsed_days"] = (pd.to_datetime(df['yyyymm'], format = "%Y%m") - pd.to_datetime(df['hl_open_date'], format = "%Y%m")).dt.days // 7
    df["account_elapsed_date"] = (pd.to_datetime(df['yyyymm'], format = "%Y%m") - pd.to_datetime(df['account_open_date'], format = "%Y%m")).dt.days // 7

    # gidごとの時系列順に番号を付けた新しいカラムを追加
    df = df.sort_values(['gid','yyyymm'],ascending=True)
    df['order'] = df.groupby('gid').cumcount() + 1

    #総収入と総支出にまとめる
    df['income'] = df['in_amt_atm'] + df['in_amt_ft'] + df['in_amt_auto']
    df['expenditure'] = df['out_amt_atm'] + df['out_amt_ft'] + df['out_amt_auto'] + df['hl_pmt'] + df['ln_pmt']

    #預貸率 = 貸出残高/預金残高
    df['deposit_ratio'] = df['hl_balance']/(df['eb_amt_jpycasa'] + df['eb_amt_jpytd'])
    #年収倍率 = 貸出残高/入金金額
    df['annual_income'] = df['hl_balance']/df['income']

    df = df.replace([np.inf,-np.inf],np.nan)

    df['deposit_ratio'] = df['deposit_ratio'].fillna(df['deposit_ratio'].median())
    df['annual_income'] = df['annual_income'].fillna(df['annual_income'].median())

    #毎月の利益を算出す
    df['profit'] = df['income'] - df['expenditure']

    #傾きの特徴量をmerge
    income = read_db('income_table_20230209')
    expenditure = read_db('expenditure_table_20230209')
    eb_amt_jpycasa = read_db('eb_amt_jpycasa_table_20230210')

    df = df.merge(income[['gid','yyyymm','income_slope','income_angle','income_profit_cumsum']],left_on=['gid','yyyymm'],right_on=['gid','yyyymm'],how='left')
    df = df.merge(expenditure[['gid','yyyymm','expenditure_slope','expenditure_angle','expenditure_profit_cumsum']],left_on=['gid','yyyymm'],right_on=['gid','yyyymm'],how='left')
    df = df.merge(eb_amt_jpycasa[['gid','yyyymm','eb_amt_jpycasa_slope','eb_amt_jpycasa_angle','eb_amt_jpycasa_profit_cumsum']],left_on=['gid','yyyymm'],right_on=['gid','yyyymm'],how='left')

    df['income_deficit_ratio'] = df['income_profit_cumsum'] / df['order']
    df['expenditure_deficit_ratio'] = df['expenditure_profit_cumsum'] / df['order']
    df['eb_amt_jpycasa'] = df['eb_amt_jpycasa_profit_cumsum'] / df['order']

    #使わないカラムをすべて削除
    delete_cols = ['out_d_atm','in_d_atm','out_d_ft','in_d_ft','out_d_auto','in_d_auto','ln_contract','ln_balance','ln_delay_flag','ln_pmt','sex',#特に使えない特徴量
                   'order','income_profit_cumsum','expenditure_profit_cumsum','eb_amt_jpycasa_profit_cumsum',#傾きのデータで不必要になったもの
                   'out_mindate_atm','out_maxdate_atm','in_mindate_atm','in_maxdate_atm','out_mindate_ft','out_maxdate_ft','in_mindate_ft','in_maxdate_ft','out_mindate_auto','out_maxdate_auto','in_mindate_auto','in_maxdate_auto']#最小日最大日は削除
    for col in delete_cols:
        df.drop(col,axis=1,inplace=True)
    return df

#型を自動変換して、メモリを減らす
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

#直近3カ月のデータを結合するための関数
def filtered_df(df,n_months):
    if n_months == 'all':
        return df.sort_values('yyyymm',ascending=False)
    else:
        return df.sort_values('yyyymm',ascending=False).groupby('gid').head(n_months)

def summarize_sum(df,target_col):
    return df.groupby('gid')[[target_col]].sum()

def summarize_mean(df,target_col):
    return df.groupby('gid')[[target_col]].mean()

def merged_past(df,n_month, target_cols):
     #空のdict作成
    output_dict = {}
    separated_dict = {}
    separated_past_dict = {}

    df = df.sort_values('yyyymm',ascending=True)
    for date,df_by_date in tqdm(df.groupby('yyyymm')):
        separated_dict[date] = df_by_date
        separated_past_dict[date] = df.query('yyyymm < @date')

    for date in tqdm(list(separated_dict)):
        recent = separated_dict[date].copy()
        past = separated_past_dict[date].copy()
        #直近nカ月の口座状態を平均する
        for month in n_months:
            if month == 'all':
                for col in target_cols:
                    past_months = summarize_sum(filtered_df(past,month),col).rename(columns={col:'{}_total'.format(col)})
                    recent = recent.merge(past_months,left_on='gid',right_on='gid',how='left')
                for col in target_cols:
                    past_months = summarize_mean(filtered_df(past,month),col).rename(columns={col:'{}_avg'.format(col)})
                    recent = recent.merge(past_months,left_on='gid',right_on='gid',how='left')
            else:
                for col in target_cols:
                    past_months = summarize_mean(filtered_df(past,month),col).rename(columns={col:'{}_avg_{}M'.format(col,month)})
                    recent = recent.merge(past_months,left_on='gid',right_on='gid',how='left')
            output_dict[date] = recent
        del separated_dict[date],separated_past_dict[date],recent,past,past_months
        gc.collect()
    data = pd.concat([output_dict[date] for date in output_dict])
    del output_dict
    gc.collect()
    return data

def slope_engineerring(data,col):
    test = data[['gid','yyyymm',col]].sort_values(['gid','yyyymm'])

    #その時点～直近2カ月前のデータから傾きを求める
    ###EDAの結果、口座残高の減り方が急な契約者は延滞しやすい傾向を見つけた為、
    ###残高の減りを方を、グラフ（二次関数）の傾きとして表現
    concat = {}
    for gid,df_test in tqdm(test.groupby('gid')):

        test= df_test.sort_values(['gid','yyyymm'])[[col]]

        df_concat = pd.concat([
                    df_test,
                    test.shift(1).rename(columns = {col:'{}_1M'.format(col)}),
                    test.shift(2).rename(columns = {col:'{}_2M'.format(col)})
                   ],axis=1).reset_index(drop=True)

        # 新しいカラムを追加する
        df_concat["{}slope".format(col)] = np.nan

        list = df_concat.index.values
        # 傾きを求めて、各行に追加する
        for i in list:
            x = np.array([200000, 100000, 0])
            y = np.array([df_concat.iloc[i][col], df_concat.iloc[i]["{}_1M".format(col)], df_concat.iloc[i]["{}_2M".format(col)]])
            mask = ~np.isnan(y)
            if np.sum(mask) >= 2:
                a, b = np.polyfit(x[mask], y[mask], 1)
                df_concat.at[i, "{}_slope".format(col)] = a
                df_concat['{}_angle'.format(col)] = round(np.arctan(df_concat['{}_slope'.format(col)])*(180/np.pi),2)
                df_concat['{}_profit'.format(col)] = df_concat['{}_angle'.format(col)].apply(lambda x:1 if x<0 else 0)
                df_concat['{}_profit_cumsum'.format(col)] = np.cumsum(df_concat['{}_profit'.format(col)])
        concat[gid] = df_concat
    df = pd.concat(concat[gid] for gid in concat )
    return df

def features_engineering(row):
    df = row.copy()
    #特徴量として弱いカラムを削除
    delete_cols = [
                  'out_n_atm','in_n_auto','out_n_ft','in_n_ft'#merged_pastで外せなかったものをここで削除する
                  ]
    for col in delete_cols:
        df = df.drop(col,axis=1)
    #各ユーザーの初月のデータを削除
    df = df[df['income_angle'].notnull()]
    #NaNの処理を行っとく
    #df = df.fillna(0)

    return reduce_mem_usage(df)


#学習するためのデータに分ける
def split_data(df,recent,past):
    train = df[(df['yyyymm']!=recent)&(df['yyyymm']>past)]
    test  = df[df['yyyymm']==recent]
    return train,test

In [ ]:
#target_colmnsと前月と累計の設定場所
n_months = [3,'all']
target_cols = [
'eb_amt_jpycasa',
'income',
'expenditure',
'in_amt_atm',
'in_amt_ft',
'in_amt_auto',
'in_n_atm',
'in_n_ft',
'in_n_auto',
'out_amt_atm',
'out_amt_ft',
'out_amt_auto',
'out_n_atm',
'out_n_ft',
'out_n_auto',
]

## データ前処理

In [ ]:
#過去データを結合
data = merged_past(data,n_months,target_cols)

#最後のデータ加工(#特徴量エンジニアリング)
data = features_engineering(data)

#加工したデータをもう一度testデータとtrainデータに分ける
data,subject = split_data(data,202111,201908)
#日付を最新順にする
data = data.sort_values('yyyymm',ascending=True)

## モデルの構築

### ➀ハイパーパラメータの最適化

In [ ]:
#validまで分ける
sample = data
train, test = split_data(sample,recent=202110,past=201908)
train, valid = split_data(train,recent=202109,past=201908)
#Optunaのためにvalidを含めた変換を行う
X_train = train.drop(['gid','yyyymm','target_flag'],axis=1)
y_train = train['target_flag']
X_test = test.drop(['gid','yyyymm','target_flag'],axis=1)
y_test = test['target_flag']
X_valid = valid.drop(['gid','yyyymm','target_flag'],axis=1)
y_valid = valid['target_flag']

lgb_train = lgb_o.Dataset(X_train.values,y_train.values)
lgb_valid = lgb_o.Dataset(X_valid.values,y_valid.values)

scale_pos_weight = ((y_train==0).count()) / ((y_train==1).count())

params = {
    'objective':'binary',
    'scale_pos_weight':scale_pos_weight,
    'random_state':100,
    'metric': 'auc',
}

lgb_clf_o = lgb_o.train(params,lgb_train,
                       valid_sets=(lgb_train,lgb_valid),
                       verbose_eval=100,
                       early_stopping_rounds=10,
                       optuna_seed=100 # optunaのseed固定
                       )

### ②クロスバリデーションを行いながらハイパーパラメータ最適化

In [ ]:
from lightgbm import LGBMRegressor, LGBMClassifier, Booster
def objective(trial):
    # lightgbm用のハイパーパラメータ
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 0.1),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.9, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0.8, 1.5),

        'min_child_samples': trial.suggest_int('min_child_samples',15,20),
        'scale_pos_weight':trial.suggest_int('scale_pos_weight',80,99),
    }

    # 時系列データのクロスバリデーション
    time_series_cv = TimeSeriesSplit(n_splits=25)
    for train_index, test_index in time_series_cv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        scale_pos_weight = ((y_train==0).sum()) / ((y_train==1).sum())

        # LGBMClassifierを学習
        model = LGBMClassifier(random_state=100,**params)
        model.fit(X_train, y_train)

        # 評価
        y_pred = model.predict_proba(X_test)[:,1]
        auc = roc_auc_score(y_test, y_pred)

        # 評価結果を返す
        return auc

#データを分ける
sample = data
train, test = split_data(sample,recent=202110,past=201908)

X = train.drop(['yyyymm','target_flag'],axis=1)
y = train['target_flag']

# 最適なハイパーパラメータを探索
study = optuna.create_study()
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータを表示
print("Best hyperparameters: ", study.best_params)


## 通常のモデル作成（ホールドアウト検証）

In [ ]:
sample = data
#モデルを作る
train,test = split_data(sample,recent=202110,past=201909)
X_train = train.drop(['gid','yyyymm','target_flag'],axis=1)
y_train = train['target_flag']
X_test = test.drop(['gid','yyyymm','target_flag'],axis=1)
y_test = test['target_flag']

params={
'objective': 'binary',
 'scale_pos_weight': 1.0,
 'random_state': 100,
 'metric': 'auc',
 'feature_pre_filter': False,
 'lambda_l1': 8.92380314653473,
 'lambda_l2': 0.0003830290292639538,
 'num_leaves': 66,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 25
}

lgb_clf = lgb.LGBMClassifier(**params)
model = lgb_clf.fit(X_train.values,y_train.values)

auc_train = roc_auc_score(
y_train,model.predict_proba(X_train)[:,1]
)

auc_test = roc_auc_score(
y_test,model.predict_proba(X_test)[:,1]
)

importance = pd.DataFrame({
    "features":X_train.columns,
    "importance":model.feature_importances_
}).sort_values("importance",ascending=False)

print('AUC: {:.3f}(train), {:.3f}(test)'.format(auc_train, auc_test))
importance[:50]

## 交差検証

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
sample = data.sort_values('yyyymm',ascending=True)

X = sample.drop(['gid','yyyymm','target_flag'],axis=1)
y = sample['target_flag']

# TimeSeriesSplitを使用してクロスバリデーションを定義する
tscv = TimeSeriesSplit(n_splits=5)

#パラメータ最適化のコピペ
params={
 'objective': 'binary',
 'scale_pos_weight': 82.22860176412937,
 'random_state': 100,
 'metric': 'auc',
 'feature_pre_filter': False,
 'lambda_l1': 1.0517138394360073,
 'lambda_l2': 7.635176818135586e-07,
 'num_leaves': 33,
 'feature_fraction': 0.8,
 'bagging_fraction': 1.0,
 'bagging_freq': 0
}

# 分割されたデータを使って学習と評価
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # LGBMClassifierのモデル生成
    lgb_train = lgb.Dataset(X_train, y_train)
    model = lgb.LGBMClassifier(**params)

    # モデルの訓練
    model.fit(X_train, y_train)

    # 予測と評価
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    print("Accuracy:", accuracy)
    print("AUC:", auc)

importance = pd.DataFrame({
    "features":X.columns,
    "importance":model.feature_importances_
}).sort_values("importance",ascending=False)
importance[:50]

## 予測

In [ ]:
y_pred = model.predict_proba(subject.drop(['gid','yyyymm','target_flag'],axis=1))[:, 1]
subject['prob'] = y_pred